# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [23]:
import pandas as pd
file_path = 'c:\\Users\\yezhu\dsi\\scaling_to_production\\05_src\\data\\fire\\forestfires.csv'
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv(file_path, header = None, names = columns))


# Get X and Y

Create the features data frame and target data.

In [24]:
# Handle any remaining missing values by imputing or dropping
fires_dt = fires_dt.dropna()

# Recheck data types and missing values
print(fires_dt.dtypes)
print(fires_dt.isnull().sum())

coord_x    object
coord_y    object
month      object
day        object
ffmc       object
dmc        object
dc         object
isi        object
temp       object
rh         object
wind       object
rain       object
area       object
dtype: object
coord_x    0
coord_y    0
month      0
day        0
ffmc       0
dmc        0
dc         0
isi        0
temp       0
rh         0
wind       0
rain       0
area       0
dtype: int64


In [25]:

# Convert numerical columns to numeric type, coercing errors
for col in numerical_cols:
    fires_dt[col] = pd.to_numeric(fires_dt[col], errors='coerce')

# Handle any remaining missing values by imputing or dropping
fires_dt = fires_dt.dropna()

In [26]:
X = fires_dt.drop('area', axis=1)
y = fires_dt['area']

In [27]:
print(X.head())
print(y.head())

   coord_x  coord_y month  day  ffmc   dmc     dc  isi  temp    rh  wind  rain
1      7.0      5.0   mar  fri  86.2  26.2   94.3  5.1   8.2  51.0   6.7   0.0
2      7.0      4.0   oct  tue  90.6  35.4  669.1  6.7  18.0  33.0   0.9   0.0
3      7.0      4.0   oct  sat  90.6  43.7  686.9  6.7  14.6  33.0   1.3   0.0
4      8.0      6.0   mar  fri  91.7  33.3   77.5  9.0   8.3  97.0   4.0   0.2
5      8.0      6.0   mar  sun  89.3  51.3  102.2  9.6  11.4  99.0   1.8   0.0
1    0
2    0
3    0
4    0
5    0
Name: area, dtype: object


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [28]:
# preproc1
numerical_cols = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_cols = ['month', 'day']

In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Preprocessor 1
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ]
)

# Inspect preproc1
print(preproc1)

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat', OneHotEncoder(), ['month', 'day'])])


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [30]:
# preproc2
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
import numpy as np
log_transformer = FunctionTransformer(np.log1p, validate=True)

# Preprocessor 2
preproc2 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('log', log_transformer, ['dc']),  
        ('cat', OneHotEncoder(), categorical_cols)
    ]
)

# Inspect preproc2
print(preproc2)

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('log',
                                 FunctionTransformer(func=<ufunc 'log1p'>,
                                                     validate=True),
                                 ['dc']),
                                ('cat', OneHotEncoder(), ['month', 'day'])])


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [31]:
# Pipeline A = preproc1 + baseline
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
import numpy as np
pipeline_A = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', LinearRegression())
])

In [32]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', LinearRegression())
])


In [33]:
# Pipeline C = preproc1 + advanced model
pipeline_C = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [34]:
# Pipeline D = preproc2 + advanced model
pipeline_D = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [35]:
param_grids = {
    'Pipeline A': {
        'regressor__fit_intercept': [True, False]
    },
    'Pipeline B': {
        'regressor__fit_intercept': [True, False]
    },
    'Pipeline C': {
        'regressor__n_estimators': [100, 200, 300],
        'regressor__max_depth': [None, 10, 20]
    },
    'Pipeline D': {
        'regressor__n_estimators': [100, 200, 300],
        'regressor__max_depth': [None, 10, 20]
    }
}


In [36]:
pipelines = {
    'Pipeline A': pipeline_A,
    'Pipeline B': pipeline_B,
    'Pipeline C': pipeline_C,
    'Pipeline D': pipeline_D
}

In [37]:
best_models = {}
best_scores = {}

for name, pipeline in pipelines.items():
    grid_search = GridSearchCV(pipeline, param_grids[name], cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X, y)
    best_models[name] = grid_search.best_estimator_
    best_scores[name] = -grid_search.best_score_
    print(f"{name} best score: {-grid_search.best_score_}")
    print(f"{name} best parameters: {grid_search.best_params_}")

best_pipeline_name = min(best_scores, key=best_scores.get)
best_model = best_models[best_pipeline_name]

c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users

Pipeline A best score: nan
Pipeline A best parameters: {'regressor__fit_intercept': True}


c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users

Pipeline B best score: nan
Pipeline B best parameters: {'regressor__fit_intercept': True}


c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users

Pipeline C best score: nan
Pipeline C best parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 100}


c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "c:\Users\yezhu\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users

Pipeline D best score: nan
Pipeline D best parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 100}


# Evaluate

+ Which model has the best performance?

In [38]:
print(f"Best performing model: {best_pipeline_name} with MSE: {best_scores[best_pipeline_name]}")

Best performing model: Pipeline A with MSE: nan


# Export

+ Save the best performing model to a pickle file.

In [40]:
import pickle
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

print(f"Best model saved as 'best_model.pkl'")

Best model saved as 'best_model.pkl'


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.